# Impala Project - Vector Search
- Impala có rất nhiều các ưu điểm như: Hiệu suất cao, Xử lý song song khối lượng lớn, là công cụ SQL có hiệu suất cao nhất (mang lại trải nghiệm giống RDBMS), cung cấp cách nhanh nhất để truy cập dữ liệu được lưu trữ trong Hệ thống tệp phân tán Hadoop,...
- Lợi dụng những ưu điểm đó Project này sẽ sử dụng Impala để làm Query engine để mô phỏng Vector Search cho một hệ thống gợi ý anime cho user
- Các thao tác trong Project này bao gồm các bước sau:


**1. Kết nối với Impala bằng gói thư viện Impyla**

**2. Vector hóa dữ liệu, chuyển thành file parquet và đẩy vào database**

**3. Xây dựng hệ thống truy vấn so sánh vector - Vector Search**
![Ảnh minh họa Apacha Impala](https://editor.analyticsvidhya.com/uploads/58241impala-800x445.jpg)

## 1. Kết nối với Impala bằng gói thư viện Impyla
- Ta cần setup một máy ảo Cloudera Distribution for Hadoop (có hỗ trợ Impala)
- Tiếp theo ta cần biết được địa chỉ IP của máy ảo đó
- Cổng mặc định của Impala là 21050

In [1]:
from impala.dbapi import connect
IP = '192.168.1.65'

In [2]:
# Test kết nối
try:
    conn = connect(host=IP)

    # Tạo một đối tượng cursor để thực thi truy vấn
    cursor = conn.cursor()

    # Thực hiện truy vấn SQL để liệt kê các cơ sở dữ liệu
    cursor.execute('SHOW DATABASES')

    # Lấy kết quả truy vấn
    databases = cursor.fetchall()

    # In danh sách các cơ sở dữ liệu
    for db in databases:
        print(db[0])

    # Đóng kết nối
    cursor.close()
    conn.close()
    print("Kết nối thành công")
except Exception as err:
    print("Lỗi kết nối!")

_impala_builtins
default
Kết nối thành công


### Thiết kế hàm Query

In [26]:
def Impala_Query(query,db_name=""):
    try:
        conn = connect(host=IP)
        cursor = conn.cursor()

        # Thực hiện truy vấn SQL để liệt kê các cơ sở dữ liệu
        if (db_name!= ""):
            cursor.execute(f"USE {db_name}")
        cursor.execute(query)
        
        # Lấy kết quả truy vấn
        result = cursor.fetchall()

        # Đóng kết nối
        cursor.close()
        conn.close()
        return result
    except:
        return None

In [4]:
Impala_Query("SHOW TABLES","default")

[]

## 2. Vector hóa dữ liệu, chuyển thành file parquet và đẩy vào database
- Tóm tắt thao tác
| Bước thao tác | Tóm tắt |
|------|---------|
| 1 | data (csv) -> (Vector hóa) -> vectors |
| 2 | vectors -> (Ghi file) -> file parquet |
| 3 | File parquet -> (đẩy vào máy ảo và tải lên hdfs thông qua phương thức ssh) |
| 4 | Tạo external table theo file parquet có trên hdfs |



- Cấu trúc database: vectordb


> - **Bảng anime_vector:** Dùng để lưu trữ các vector (index và value của vector) đại diện cho content của các bộ phim anime được sử dụng cho vector search xây dựng hệ thống gợi ý Content-Based Collaborative Filtering Recommend System. Ngoài ra ta có thể lưu trữ ở dạng complex type như ARRAY Nhưng vì máy tính cá nhân có giới hạn về phần cứng cho nên nếu đặt kiểu dữ liệu ARRAY thì truy vấn rất tốn kém tài nguyên máy. Cho nên ở đây ta sẽ chọn cách lưu trữ index và value của một vector để tối ưu nhất có thể trong việc lưu trữ vector trong ma trận thưa 


| Field  | Data Type  |
|--------|------------|
| vector_id     | INT |
| vector_index | INT |
| vector_value | INT |

```sql
CREATE EXTERNAL TABLE anime_vector (
  vector_id INT,
  vector_index INT,
  vector_value INT
)
STORED AS PARQUET
LOCATION 'hdfs://quickstart.cloudera:8020/user/hoaitrong/anime_vector_parquet';
```

> - **Bảng anime_mapping:** Dùng để lưu trữ thông tin của bộ phim anime tương ứng với vector_id


| Field      | Data Type |
|------------|-----------|
| vector_id  | INT       |
| title      | STRING    |
| img_url    | STRING    |
| link       | STRING    |

```sql
CREATE EXTERNAL TABLE anime_mapping (
  vector_id INT,
  title STRING,
  img_url STRING,
  link STRING
)
STORED AS PARQUET
LOCATION 'hdfs://quickstart.cloudera:8020/user/hoaitrong/anime_mapping_parquet';
```

### Xử lý dữ liệu và vector hóa
- Nguồn: [Anime Dataset with Reviews - MyAnimeList (Kaggle)](https://www.kaggle.com/datasets/marlesson/myanimelist-dataset-animes-profiles-reviews)
- Tên file sử dụng: animes.csv
- Mô tả: Tập dữ liệu "animes.csv" chứa thông tin về các bộ anime, bao gồm các trường dữ liệu quan trọng như tiêu đề, các đồng nghĩa của tiêu đề, thể loại, thời lượng, xếp hạng, độ phổ biến, điểm số, ngày phát sóng, số tập và nhiều thông tin khác. Dữ liệu này cung cấp đủ thông tin để phân tích các xu hướng theo thời gian về các khía cạnh quan trọng của anime.

In [3]:
# import Library
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

C:\Users\ADMIN\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\ADMIN\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
# import dataset
df = pd.read_csv("./csv/animes.csv")
df.drop_duplicates(inplace=True)
df.head(5)

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


In [5]:
df.shape

(16368, 12)

- Ở đây em chọn content đặc trưng cho anime là bao gồm synopsis (tóm tắt) và genre (thể loại) để vector hóa nó
- Xử lý dữ liệu:
> - Đối với **genre**: Ta loại bỏ các ký tự đặc biệt
> - Đối với **synopsis**: Ta sẽ fillna và sử dụng Rake - một thư viện xử lý ngôn ngữ để trích lọc các từ khóa quan trọng có trong đoạn văn bản tóm tắt

In [6]:
syp = df['synopsis']
genre = df['genre']

In [7]:
# Xử lí genre
genre = genre.str.replace('[','')
genre = genre.str.replace(']','')
genre = genre.str.replace("'","")
genre = genre.str.replace(",","")
genre

0                       Comedy Sports Drama School Shounen
1                       Drama Music Romance School Shounen
2                   Sci-Fi Adventure Mystery Drama Fantasy
3        Action Military Adventure Comedy Drama Magic F...
4                      Action Mystery Supernatural Vampire
                               ...                        
19002       Action Comedy Super Power Martial Arts Shounen
19003                    Slice of Life Comedy Supernatural
19004                         Slice of Life Comedy Shounen
19005                                               Action
19006            Comedy Drama Romance School Slice of Life
Name: genre, Length: 16368, dtype: object

In [8]:
# Xử lí syp
syp = syp.fillna(' ')

# Sử dụng thư viện rake_nltk để rút trích từ khóa quan trọng có trong synopsis
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')

rake = Rake()
keywords = []
for plot in syp:
    rake.extract_keywords_from_text(plot)
    keywords_i = rake.get_ranked_phrases()
    keywords_i_string = ""
    for keyword in keywords_i:
        keywords_i_string = keywords_i_string + " " + keyword
    keywords.append(keywords_i_string)
df['keywords'] = keywords
df['keywords'][0]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


' large training camp alongside many notable volleyball teams karasuno high school volleyball team attempts volleyball team must learn overcome formidable opponents old class setter tooru oikawa standing rival nekoma high spring tournament instead senior players graduate national level players could possibly break archrival aoba jousai new — including toughest teams karasuno agrees new attacks would strengthen train harder take part powerful weapon mal rewrite last chance kageyama attempt also come high karasuno written world victory tokyo sturdiest skills sharpen settle refocus receive playing participation one moreover members may long japan invitation inter hope hope hinata following facing ever even efforts differences devise conquer blocks aiming'

In [9]:
content = genre + df['keywords']

### Vector hóa content

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
vectorizer = CountVectorizer()
vectorized_content_based = vectorizer.fit_transform(content)
vectorized_content_based = vectorized_content_based.toarray()
vectorized_content_based

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### Đẩy dữ liệu vector và anime vào vectordb bằng file parquet
- Đầu tiên ta sẽ tạo ra các file parquet. Sau đó truyền file, đẩy file lên hdfs và thực hiện load data

In [12]:
# vector
count = 1
id = []
indexes = []
values = []
for vector in vectorized_content_based:
    for i,val in enumerate(vector):
        if val != 0:
            id.append(count)
            indexes.append(i)
            values.append(val)
    count+=1
vector_df = pd.DataFrame({
    'vector_id': id,
    'vector_index': indexes,
    'vector_value': values
})


# mapping
mapping_df = pd.DataFrame({
    'vector_id': list(range(1, len(vectorized_content_based) + 1)),
    'title': df['title'],
    'img_url': df['img_url'],
    'link':df['link']
})

# convert type
vector_df['vector_id'] = vector_df['vector_id'].astype(np.int32)
vector_df['vector_index'] = vector_df['vector_index'].astype(np.int32)
vector_df['vector_value'] = vector_df['vector_value'].astype(np.int32)
mapping_df['vector_id'] = mapping_df['vector_id'].astype(np.int32)

In [13]:
import pyarrow as pa
import pyarrow.parquet as pq

In [14]:
# Tạo schema và table vector
vector_schema = pa.schema([
    ('vector_id', pa.int32()),
    ('vector_index', pa.int32()),
    ('vector_value', pa.int32()),
])
vector_table = pa.Table.from_pandas(vector_df, schema=vector_schema)

# Tạo schema và table mapping
mapping_schema = pa.schema([
    ('vector_id', pa.int32()),
    ('title', pa.string()),
    ('img_url', pa.string()),
    ('link', pa.string())
])
mapping_table = pa.Table.from_pandas(mapping_df, schema=mapping_schema)

In [15]:
# Lưu table thành file parquet
pq.write_table(vector_table, 'vector.parquet',version="1.0")
pq.write_table(mapping_table, 'mapping.parquet',version="1.0")

- Truyền file và đẩy file lên hdfs

In [16]:
import paramiko
import os

In [17]:
def transfer_file_to_centos(local_path, remote_path, centos_ip, username, password):
    # Tạo một đối tượng SSHClient từ thư viện Paramiko
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    try:
        # Kết nối đến máy CentOS
        ssh.connect(centos_ip, username=username, password=password)
        
        # Sử dụng SFTP để truyền tệp
        sftp = ssh.open_sftp()
        
        # Truyền tệp từ máy Windows sang máy CentOS
        sftp.put(local_path, remote_path)
        
        # Đóng kết nối SFTP
        sftp.close()
        
        print("File đã được truyền thành công!")
    except Exception as e:
        print("Lỗi khi truyền tệp:", e)
    finally:
        # Đóng kết nối SSH
        ssh.close()
        
def command_line_to_centos(centos_ip, username, password, command):
    # Tạo một đối tượng SSHClient từ thư viện Paramiko
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    try:
        # Kết nối đến máy CentOS
        ssh.connect(centos_ip, username=username, password=password)
        
        # Thực hiện lệnh trên máy CentOS
        stdin, stdout, stderr = ssh.exec_command(command)
        
        # In ra kết quả của lệnh
        print(stdout.read().decode())
        print(stderr.read().decode())
    except Exception as e:
        print("Lỗi khi thực hiện lệnh:", e)
    finally:
        # Đóng kết nối SSH
        ssh.close()


In [18]:
# Thông tin đăng nhập cho máy CentOS
centos_ip = IP
username = "cloudera"
password = "cloudera"

In [19]:
# Tạo ra thư mục chứa file parquet cũng như cấu trúc metadata của parquet đó (bên máy centos)
command_line_to_centos(centos_ip,username,password,"mkdir /home/cloudera/anime_vector_parquet")
command_line_to_centos(centos_ip,username,password,"mkdir /home/cloudera/anime_mapping_parquet")

# Đẩy 2 file parquet vào 2 thư mục trên
transfer_file_to_centos("./vector.parquet","/home/cloudera/anime_vector_parquet/vector.parquet",centos_ip,username,password)
transfer_file_to_centos("./mapping.parquet","/home/cloudera/anime_mapping_parquet/mapping.parquet",centos_ip,username,password)





File đã được truyền thành công!
File đã được truyền thành công!


In [20]:
# Copy 2 thư mục đã tạo vào hdfs
command_line_to_centos(centos_ip,username,password,"hdfs dfs -copyFromLocal /home/cloudera/anime_mapping_parquet hdfs://quickstart.cloudera:8020/anime_mapping_parquet")
command_line_to_centos(centos_ip,username,password,"hdfs dfs -copyFromLocal /home/cloudera/anime_vector_parquet hdfs://quickstart.cloudera:8020/anime_vector_parquet")


copyFromLocal: Permission denied: user=cloudera, access=WRITE, inode="/":hdfs:supergroup:drwxr-xr-x


copyFromLocal: Permission denied: user=cloudera, access=WRITE, inode="/":hdfs:supergroup:drwxr-xr-x



#### Lưu ý: nếu có lỗi sau
```
copyFromLocal: Permission denied: user=cloudera, access=WRITE, inode="/":hdfs:supergroup:drwxr-xr-x
```
- Ta sẽ giải quyết bằng cách tạo ra một thư mục user mới trong hdfs và cấp quyền cho nó

In [21]:
# Giải quyết lỗi Permission denied
command_line_to_centos(centos_ip,username,password,"sudo -u hdfs hadoop fs -mkdir /user/hoaitrong")
command_line_to_centos(centos_ip,username,password,"sudo -u hdfs hadoop fs -chmod -R 777 /user/hoaitrong")


mkdir: `/user/hoaitrong': File exists





In [22]:
command_line_to_centos(centos_ip,username,password,"hdfs dfs -ls hdfs://quickstart.cloudera:8020/user/")

Found 9 items
drwxr-xr-x   - cloudera cloudera            0 2024-05-11 02:31 hdfs://quickstart.cloudera:8020/user/cloudera
drwxr-xr-x   - mapred   hadoop              0 2024-05-11 01:38 hdfs://quickstart.cloudera:8020/user/history
drwxrwxrwx   - hive     supergroup          0 2017-10-23 09:17 hdfs://quickstart.cloudera:8020/user/hive
drwxrwxrwx   - hdfs     supergroup          0 2024-05-14 07:01 hdfs://quickstart.cloudera:8020/user/hoaitrong
drwxrwxrwx   - hue      supergroup          0 2024-05-11 01:44 hdfs://quickstart.cloudera:8020/user/hue
drwxrwxrwx   - jenkins  supergroup          0 2017-10-23 09:15 hdfs://quickstart.cloudera:8020/user/jenkins
drwxrwxrwx   - oozie    supergroup          0 2017-10-23 09:16 hdfs://quickstart.cloudera:8020/user/oozie
drwxrwxrwx   - root     supergroup          0 2024-05-11 11:02 hdfs://quickstart.cloudera:8020/user/root
drwxr-xr-x   - hdfs     supergroup          0 2017-10-23 09:17 hdfs://quickstart.cloudera:8020/user/spark




In [23]:
command_line_to_centos(centos_ip,username,password,"hdfs dfs -copyFromLocal /home/cloudera/anime_mapping_parquet hdfs://quickstart.cloudera:8020/user/hoaitrong/anime_mapping_parquet")
command_line_to_centos(centos_ip,username,password,"hdfs dfs -copyFromLocal /home/cloudera/anime_vector_parquet hdfs://quickstart.cloudera:8020/user/hoaitrong/anime_vector_parquet")

In [24]:
command_line_to_centos(centos_ip,username,password,"hdfs dfs -ls hdfs://quickstart.cloudera:8020/user/hoaitrong/")

Found 2 items
drwxr-xr-x   - cloudera supergroup          0 2024-05-14 08:55 hdfs://quickstart.cloudera:8020/user/hoaitrong/anime_mapping_parquet
drwxr-xr-x   - cloudera supergroup          0 2024-05-14 08:55 hdfs://quickstart.cloudera:8020/user/hoaitrong/anime_vector_parquet




### Tạo database và tạo bảng

In [27]:
# Tạo csdl vectordb
Impala_Query("CREATE DATABASE IF NOT EXISTS vectordb")
databases = Impala_Query("SHOW DATABASES")
for db in databases:
    print(db[0])

_impala_builtins
default
vectordb


In [28]:
# Tạo bảng anime_vector
query = """
CREATE EXTERNAL TABLE anime_vector (
    vector_id INT, 
    vector_index INT,
    vector_value INT
) 
STORED AS PARQUET 
LOCATION 'hdfs://quickstart.cloudera:8020/user/hoaitrong/anime_vector_parquet';
"""
Impala_Query(query,"vectordb")

query = """
CREATE EXTERNAL TABLE anime_mapping (
    vector_id INT, 
    title STRING, 
    img_url STRING, 
    link STRING
) 
STORED AS PARQUET 
LOCATION 'hdfs://quickstart.cloudera:8020/user/hoaitrong/anime_mapping_parquet'
"""
Impala_Query(query,"vectordb")

In [29]:
# Kiểm tra cấu trúc
print("Cấu trúc bảng anime_vector:")
result = Impala_Query("DESCRIBE anime_vector","vectordb")
for row in result:
    print(row[0:2])
    
print("Cấu trúc bảng anime_mapping:")
result = Impala_Query("DESCRIBE anime_mapping","vectordb")
for row in result:
    print(row[0:2])

Cấu trúc bảng anime_vector:
('vector_id', 'int')
('vector_index', 'int')
('vector_value', 'int')
Cấu trúc bảng anime_mapping:
('vector_id', 'int')
('title', 'string')
('img_url', 'string')
('link', 'string')


### Thử truy vấn đơn giản

In [30]:
import time

In [31]:
start = time.time()
result = Impala_Query("SELECT * FROM anime_mapping WHERE title='Naruto'","vectordb")
end = time.time()
print(result)
print(end - start," seconds")

[(145, 'Naruto', 'https://cdn.myanimelist.net/images/anime/13/17405.jpg', 'https://myanimelist.net/anime/20/Naruto')]
0.9109818935394287  seconds


In [32]:
start = time.time()
result = Impala_Query("SELECT * FROM anime_vector","vectordb")
end = time.time()
print(f"Truy vấn {len(result)} dòng trong thời gian {end - start} giây!")

Truy vấn 575171 dòng trong thời gian 1.3557868003845215 giây!


## 3. Xây dựng hệ thống truy vấn so sánh vector - Vector Search

- Vector Search trong Project được thực hiện như sau:
> - **Người dùng** -------- Thực hiện truy vấn theo tên anime --------------> **Vector Seach Program**
> - **Vector Seach Program** ------- Lấy ra vector tương ứng với tên anime ----> **vectordb**
> - **Vector Seach Program** --- Lấy ra toàn bộ vector ----> **vectordb**
> - **Vector Seach Program** --- Tính toán N các vector tương tự --- Lấy ra thông tin anime mapping tương ứng ---> **vectordb**
> - **Vector Seach Program** --- Trả về thông tin top anime tương tự -------> **Người dùng**

In [33]:
import math

In [34]:
class Vector:
    def __init__(self,vector_id, vector_inf):
        self.vector_id = vector_id
        self.vector_inf = vector_inf
        
    def length(self):
        length = 0
        for inf in self.vector_inf:
            length += inf[1] ** 2
        return math.sqrt(length)

def similar_cosine(vector_a,vector_b):
    dot_vector = 0
    for inf_a in vector_a.vector_inf:
        for inf_b in vector_b.vector_inf:
            if inf_a[0] == inf_b[0]:
                dot_vector += inf_a[1] * inf_b[1]
    return dot_vector / (vector_a.length() * vector_b.length())
    
def Vector_query(title):
    anime_mapping = Impala_Query(f"SELECT vector_id FROM anime_mapping WHERE title='{title}'","vectordb")
    if anime_mapping == None:
        print("Không có kết quả nào!")
        return None
    else:
        anime_vector = Impala_Query(f"SELECT vector_index, vector_value FROM anime_vector WHERE vector_id={anime_mapping[0][0]}","vectordb")
        return Vector(anime_mapping[0][0],anime_vector)
    
def Vector_search(vector,limit=10):
    full_anime_vector = Impala_Query(f"SELECT * FROM anime_vector","vectordb")
    vectors = []
    anime_vector_dict = {}
    for anime_vector in full_anime_vector:
        key = anime_vector[0]
        if key in anime_vector_dict:
            anime_vector_dict[key].append(anime_vector[1:3])
        else:
            anime_vector_dict[key] = [anime_vector[1:3]]
    for key in anime_vector_dict:
        vectors.append(Vector(key,anime_vector_dict[key]))
        
    sim_cosine_dict = {}
    for vector_temp in vectors:
        sim_cosine_dict[vector_temp.vector_id] = similar_cosine(vector,vector_temp)
    
    sorted_sim_cosine_dict = sorted(sim_cosine_dict.items(), key=lambda x: x[1], reverse=True)[1:limit+1]
    top_id = [inf[0] for inf in sorted_sim_cosine_dict]
    
    # Return result
    animes = Impala_Query(f"SELECT * FROM anime_mapping WHERE vector_id IN ({','.join(map(str,top_id))})","vectordb")
    for anime in animes:
        print(anime)
    

In [36]:
start = time.time()
vector = Vector_query("Naruto")
Vector_search(vector,5)
end = time.time()

print("\n\n=============================================================")
print(f"Vector Search được thực hiện trong {end-start} giây!")

(486, 'Naruto: Shippuuden', 'https://cdn.myanimelist.net/images/anime/5/17407.jpg', 'https://myanimelist.net/anime/1735/Naruto__Shippuuden')
(924, 'Naruto: Shippuuden Movie 6 - Road to Ninja', 'https://cdn.myanimelist.net/images/anime/6/51863.jpg', 'https://myanimelist.net/anime/13667/Naruto__Shippuuden_Movie_6_-_Road_to_Ninja')
(4131, 'Boruto: Naruto the Movie', 'https://cdn.myanimelist.net/images/anime/4/78280.jpg', 'https://myanimelist.net/anime/28755/Boruto__Naruto_the_Movie')
(5503, 'Naruto: Shippuuden Movie 5 - Blood Prison', 'https://cdn.myanimelist.net/images/anime/13/41403.jpg', 'https://myanimelist.net/anime/10589/Naruto__Shippuuden_Movie_5_-_Blood_Prison')
(13478, 'Boruto: Naruto Next Generations', 'https://cdn.myanimelist.net/images/anime/9/84460.jpg', 'https://myanimelist.net/anime/34566/Boruto__Naruto_Next_Generations')


Vector Search được thực hiện trong 4.815884828567505 giây!


# Kết luận:
- **Về tốc độ:**
> - Tốc độ truy vấn của Impala thật sự rất nhanh trong Demo Project này. Nếu được setup ở điều kiện vật chất tốt hơn cũng như là phát triển thêm về cách lưu trữ (phân vùng, giải thuật index tối ưu hơn,...), Impala sẽ còn truy vấn nhanh hơn mong đợi của chúng ta
> - Với 16368 thông tin anime và 575171 dòng thông tin vector phim được lưu trữ trong database thì việc truy vấn bằng các query enginee thông thường thì có lẽ rất mất thời gian hơn hẵn. Nhưng với Impala - công cụ truy vấn dữ liệu song song mạnh mẽ, thời gian truy vấn được rút ngắn đáng kể.

- **Về hiệu suất:**
> - Trong quá trình thử nghiệm, Impala đã chứng minh được khả năng xử lý dữ liệu với tốc độ đáng kinh ngạc. Điều này chứng tỏ tiềm năng lớn của công cụ này khi được triển khai trong môi trường sản xuất.
Số lượng lớn thông tin anime và dòng thông tin vector phim được lưu trữ không làm ảnh hưởng đến khả năng xử lý của Impala, điều này là một ưu điểm lớn so với các query engine thông thường.

- **Về khả năng mở rộng:**
> - Impala đã thể hiện sự linh hoạt trong việc mở rộng và điều chỉnh để phù hợp với nhu cầu và quy mô dữ liệu của dự án.
Việc tối ưu hóa cách lưu trữ, bao gồm phân vùng và sử dụng các giải thuật index hiệu quả, có thể giúp tăng cường hiệu suất của Impala trong tương lai.

- **Kế hoạch phát triển Project:**
> - Cần tiếp tục nghiên cứu và thử nghiệm các phương pháp tối ưu hóa khác nhau để tăng cường hiệu suất và khả năng mở rộng của Impala.
> - Xem xét việc triển khai các cải tiến về cách lưu trữ và quản lý dữ liệu để tối ưu hóa việc sử dụng Impala trong môi trường sản xuất.